# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
df= pd.read_csv('IdealCities.csv')
df.head()


,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,5546220,Saint George,US,37.10,-113.58,82.0,16.0,1.0,6.93,1.588529e+09
1,3874787,Punta Arenas,CL,-53.15,-70.92,48.2,81.0,51.0,13.87,1.588529e+09
2,2163355,Hobart,AU,-42.88,147.33,52.0,76.0,40.0,9.17,1.588529e+09
3,1633419,Padang,ID,-0.95,100.35,79.7,76.0,99.0,4.52,1.588530e+09
4,2208248,Kaitangata,NZ,-46.28,169.85,57.0,54.0,100.0,6.44,1.588530e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
#Store locations and humidities
locations = df[['Lat', 'Lng']]
humidity = df.Humidity

In [23]:
# Create heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=True)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
mask1 = (df['Max Temp'] >= 75) & (df['Max Temp'] < 90)
mask2 = (df['Wind Speed'] > 5) & (df['Wind Speed'] < 15)
mask3 = df.Cloudiness < 30
mask4 = mask1 & mask2 & mask3

ideal_cities = df.loc[mask4].reset_index(drop=True)
len(ideal_cities)

34

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
#Find a hotel
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

lat = ideal_cities["Lat"][0]
lng = ideal_cities["Lng"][0]

params["location"] = f"{lat},{lng}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

name_address = requests.get(base_url, params=params)
name_address = name_address.json()

name_address["results"][0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 37.0784851, 'lng': -113.5825771},
  'viewport': {'northeast': {'lat': 37.07982422989272,
    'lng': -113.5813398701073},
   'southwest': {'lat': 37.07712457010728, 'lng': -113.5840395298927}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
 'id': 'f8e82ffb65daeaa396d16d18d1718d96fae39823',
 'name': 'Hilton Garden Inn St. George',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 1366,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105535135526381524133">Hilton Garden Inn St. George</a>'],
   'photo_reference': 'CmRaAAAAUgevAG5D10_QTVFt0gm7PSB-U4OvINTsgFxKJ_bt2RAkNQuzgXH7jjI8sIr31ICBNVtC1t6RIYH6xjr9Tass3iHvd5GTOGPq9fcwPkvGHGgtYBj8jVPeF_mSWMxspD5DEhCNFXFw4pXzfcfW2NLXei7ZGhTz6_tgUPNIkcckQ6HEXjDECGcOaw',
   'width': 2048}],
 'place_id': 'ChIJl0DXzzdFyoAR96GSwl7RNYo',
 'plus_code': {'compound_code': '3CH8+9X St. George, Utah',
  'global_code': '85983CH8+9X'

In [31]:
#iterrate through all locations
names = []
addresses = []
ratings = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in ideal_cities.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # create url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    
    try:
        names.append(name_address["results"][0]["name"])
        addresses.append(name_address["results"][0]["vicinity"])
        ratings.append(name_address["results"][0]["rating"])
    except (KeyError, IndexError):
        print("No hotels listed nearby.")
        names.append("No listed hotels")
        addresses.append("")
        ratings.append("")

No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.
No hotels listed nearby.


In [34]:
ideal_cities["Hotel"] = names
ideal_cities["Address"] = addresses
ideal_cities["Rating"] = ratings
ideal_cities.head(30)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel,Address,Rating
0,5546220,Saint George,US,37.10,-113.58,82.00,16.0,1.0,6.93,1.588529e+09,Hilton Garden Inn St. George,"1731 S Convention Center Dr, St. George",4.4
1,5282297,Bethel,US,41.37,-73.41,77.00,49.0,1.0,11.41,1.588530e+09,Hampton Inn Danbury,"81 Newtown Rd, Danbury",4.4
2,4015700,Cananea,MX,30.95,-110.30,84.00,20.0,0.0,12.06,1.588530e+09,Hotel LOMA BONITA EJECUTIVO,Cananea,4.4
3,2467813,Rafraf,TN,37.19,10.18,75.20,41.0,20.0,14.99,1.588530e+09,Tomoko @ Salah.Rafraf Plage,Raf Raf,0
4,3576994,Cockburn Town,TC,21.46,-71.14,80.06,74.0,26.0,6.98,1.588530e+09,Osprey Beach Hotel,"1 Duke St, Cockburn Town",4.4
5,3394023,Natal,BR,-5.79,-35.21,86.00,70.0,20.0,13.87,1.588530e+09,Porto Suites Natal Hotel,"Av. Pres. Café Filho, 682 A - Praia dos Artist...",3.8
6,1168700,Ormara,PK,25.21,64.64,82.17,83.0,0.0,11.59,1.588530e+09,Zero Point Ormara,"Ormara Chawk Monument, Ormara",0
7,4231997,Avera,US,33.19,-82.53,84.99,37.0,1.0,12.75,1.588530e+09,No listed hotels,,
8,5495360,Tucumcari,US,35.17,-103.72,77.00,41.0,1.0,9.17,1.588530e+09,Fairfield Inn & Suites by Marriott Tucumcari,"1901 S Mountain Rd, Tucumcari",4.5
9,3988086,Saltillo,MX,25.42,-101.00,87.80,38.0,5.0,10.42,1.588530e+09,Courtyard by Marriott Saltillo,"Boulevard Galerías Saltillo No 375-1 Col, Blvd...",4.6


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
#Plot locations with marker popups

#define locations
coordinates = ideal_cities[['Lat', 'Lng']]

#store weather and city info
hotelinfo= []
for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Name</dt><dd>{row.City}</dd>
                <dt>Country</dt><dd>{row.Country}</dd>
                <dt>Temp</dt><dd>{row['Max Temp']}</dd>
                <dt>Cloudiness</dt><dd>{row.Cloudiness}%</dd>
                <dt>Hotel Name</dt><dd>{row.Hotel}</dd>
                </dl>
                """
    
    hotelinfo.append(info_box)

# Display figure


In [42]:
#make plot
fighotel = gmaps.figure(map_type="HYBRID")

# Add markers
markers = gmaps.marker_layer(coordinates, info_box_content=hotelinfo )

# Add marker layer
fighotel.add_layer(markers)

# Display figure
fighotel

Figure(layout=FigureLayout(height='420px'))